# Analyze results of Timeloop/Accelergy on NeRF

In [1]:
import json
import os
import pandas as pd
import regex as re

In [2]:
!ls profile_results

eyeriss_like_onchip_compression_results.json
eyeriss_like_results.json
eyeriss_like_shen_results.json
eyeriss_like_w_gating_results.json
results.json
simba_like_results.json
simple_weight_stationary_results.json


## Load Results from the Profile notebok

In [3]:
def load_results(results_path: str = "profile_results/results.json") -> dict:
    """ Load results from Profile NeRF.ipynb notebook back into original shape. """
    with open(results_path, "r") as f:
        results = json.load(f)
    print(f"Loaded results from {results_path}")

    for arch, arch_result_dict in results.items():
        arch_results_path = arch_result_dict["results"]
        assert os.path.exists(arch_results_path), f"Could not find {arch_results_path} for {arch}"

        with open(arch_results_path, "r") as f:
            arch_result_dict["results"] = json.load(f)    
        print(f"Populated {arch} results from {arch_results_path}")
        
    return results

profile_results = load_results()

Loaded results from profile_results/results.json
Populated eyeriss_like results from /home/workspace/notebooks/profile_results/eyeriss_like_results.json
Populated eyeriss_like_onchip_compression results from /home/workspace/notebooks/profile_results/eyeriss_like_onchip_compression_results.json
Populated eyeriss_like_w_gating results from /home/workspace/notebooks/profile_results/eyeriss_like_w_gating_results.json


## Analyze

In [4]:
# Compute the actual area as the summary got it wrong
area_regex = r"Area.*:\s(\d+.\d+)"

for arch, arch_results in profile_results.items():
    arch_total_area = 0.0
    layer_summary = arch_results["layer_summary"]
    
    for layer_id in layer_summary:
        # Read all areas from the timeloop stats file
        stats_fname = layer_summary[layer_id]["stats_fname"]
        with open(stats_fname, "r") as f:
            stats = [l.strip() for l in f.readlines()]

        area_lines = [line for line in stats[:-50] if re.match(area_regex, line)]
        areas = [re.search(area_regex, line).group(1) for line in area_lines]
        areas = [float(area) for area in areas]
        assert len(areas) == 7
        area = sum(areas)
        
        # Need to multiply by number of this layer type
        num = layer_summary[layer_id]["num"]
        total_area = area * num
        
        layer_summary[layer_id]["actual_total_area"] = total_area
        layer_summary[layer_id]["actual_area"] = area
        arch_total_area += total_area
        
    arch_results["summary"]["actual_total_area"] = arch_total_area

In [5]:
# Massage results into dataframes
layer_dfs = {}
all_summary = {}
# Remove area and total_area as they are wrong as just 0.0
drop_cols = ["name", "stats_fname", "total_area", "area"]

for arch, arch_results in profile_results.items():
    all_summary[arch] = arch_results["summary"]

    # Load layer results into dataframe
    df_layer = pd.DataFrame.from_dict(arch_results["layer_summary"], orient="index")
    df_layer = df_layer.drop(columns=drop_cols)
    df_layer.index.name = "layer_id"
    layer_dfs[arch] = df_layer
    
df_summary = pd.DataFrame.from_dict(all_summary, orient="index")
df_summary = df_summary.drop(columns=["total_area"])

In [6]:
layer_dfs["eyeriss_like"]

,total_energy,total_cycle,num,energy,cycle,gflops,utilization,edp,shape,actual_total_area,actual_area
layer_id,,,,,,,,,,,
1,11.42,10752,1,11.42,10752,190.48,0.57,0.12300,"{'C': 63, 'M': 256, 'N': 64}",1375449.72,1375449.72
2,260.40,458752,7,37.20,65536,127.75,0.38,2.44000,"{'C': 256, 'M': 256, 'N': 64}",9628148.04,1375449.72
6,59.17,59392,1,59.17,59392,175.72,0.52,3.51000,"{'C': 319, 'M': 256, 'N': 64}",1375449.72,1375449.72
9,2.84,2048,1,2.84,2048,15.97,0.05,0.00581,"{'C': 256, 'M': 1, 'N': 64}",1375449.72,1375449.72
11,315.54,36224,1,315.54,36224,127.77,0.38,11.40000,"{'C': 283, 'M': 128, 'N': 64}",1375449.72,1375449.72
12,1.56,1024,1,1.56,1024,47.81,0.14,0.00160,"{'C': 128, 'M': 3, 'N': 64}",1375449.72,1375449.72


In [7]:
layer_dfs["eyeriss_like_onchip_compression"]

,total_energy,total_cycle,num,energy,cycle,gflops,utilization,edp,shape,actual_total_area,actual_area
layer_id,,,,,,,,,,,
1,12.18,10752,1,12.18,10752,191.32,0.57,0.131000,"{'C': 63, 'M': 256, 'N': 64}",907149.12,907149.12
2,28.98,29215,1,28.98,29215,127.83,0.38,0.847000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
3,26.05,20007,1,26.05,20007,127.75,0.38,0.521000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
4,25.94,19629,1,25.94,19629,127.74,0.38,0.509000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
5,26.23,20357,1,26.23,20357,127.74,0.38,0.534000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
6,62.80,27214,1,62.80,27214,175.79,0.52,1.710000,"{'C': 319, 'M': 256, 'N': 64}",907149.12,907149.12
7,27.09,22932,1,27.09,22932,127.78,0.38,0.621000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
8,26.07,20061,1,26.07,20061,127.75,0.38,0.523000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
9,0.84,617,1,0.84,617,15.96,0.05,0.000519,"{'C': 256, 'M': 1, 'N': 64}",907149.12,907149.12


In [8]:
layer_dfs["eyeriss_like_w_gating"]

,total_energy,total_cycle,num,energy,cycle,gflops,utilization,edp,shape,actual_total_area,actual_area
layer_id,,,,,,,,,,,
1,10.10,10752,1,10.10,10752,191.32,0.57,0.109000,"{'C': 63, 'M': 256, 'N': 64}",854095.4,854095.4
2,22.50,32768,1,22.50,32768,113.97,0.34,0.737000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
3,20.48,32768,1,20.48,32768,78.00,0.23,0.671000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
4,20.42,32768,1,20.42,32768,76.52,0.23,0.669000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
5,20.61,32768,1,20.61,32768,79.36,0.24,0.675000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
6,101.98,29696,1,101.98,29696,161.09,0.48,3.030000,"{'C': 319, 'M': 256, 'N': 64}",854095.4,854095.4
7,21.13,32768,1,21.13,32768,89.42,0.27,0.692000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
8,20.50,32768,1,20.50,32768,78.21,0.23,0.672000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
9,3.13,1027,1,3.13,1027,9.59,0.03,0.003220,"{'C': 256, 'M': 1, 'N': 64}",854095.4,854095.4


In [9]:
df_summary

,total_energy,total_cycle,num_params,macs,activation_size,actual_total_area
eyeriss_like,650.93,568192.0,595844,593450,2300.0,16505396.64
eyeriss_like_onchip_compression,316.71,227813.0,595844,593450,2300.0,10885789.44
eyeriss_like_w_gating,313.05,307611.0,595844,593450,2300.0,10249144.80


In [10]:
edp = df_summary["total_energy"] * df_summary["total_cycle"]
df_summary["EDP (J*cycle)"] = edp

## Save results to LaTeX table

In [11]:
arch_to_label = {
    "eyeriss_like": "Eyeriss",
    "eyeriss_like_w_gating": "w/ Gating",
    "eyeriss_like_onchip_compression": "w/ Onchip Compression",
    "eyeriss_like_shen": "w/ Shen",
}
cols = ["Energy ($\mu\text{J}$)", "Cycles", "EDP (J*cycle)", "Area ($\text{mm}^2$)"]

df_output = df_summary.copy()
df_output.index = [arch_to_label[arch] for arch in df_output.index.values]

df_output["Energy ($\mu\text{J}$)"] = df_output["total_energy"]
df_output["Cycles"] = df_output["total_cycle"].astype(int)
# Convert area in um^2 to mm^2
df_output["Area ($\text{mm}^2$)"] = round(df_output["actual_total_area"] / 1e6, 2)

df_output = df_output[cols]
df_output

,Energy ($\mu\text{J}$),Cycles,EDP (J*cycle),Area ($\text{mm}^2$)
Eyeriss,650.93,568192,3.698532e+08,16.51
w/ Onchip Compression,316.71,227813,7.215066e+07,10.89
w/ Gating,313.05,307611,9.629762e+07,10.25


In [1]:
print(df_output.to_latex())

NameError: name 'df_output' is not defined